<a href="https://colab.research.google.com/github/priyadharsh73/airbnb_eda/blob/main/airbnb_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>